**Here is the implementation of autocorrect system for suggesting the possible correct spelling or correcting the spelling mistake. Here I used the NLTK Library and machine learning for predicting the correct spelling for the given input word.**

In [25]:
import re   # importing regular expression

# words
w = []  #Initialising to store the word

# reading text file
with open('data.txt', 'r', encoding="utf8") as f: # open the data.text file in read mode
	file_name_data = f.read() #Real the content of the file
	file_name_data = file_name_data.lower() #converts the entire dataset in lower case
	w = re.findall('\w+', file_name_data) #Uses the re.findall function to find all sequences of word characters
	                                      # (letters, digits, and underscores) in the text.
                                        # The result is a list of words stored in w

# vocabulary
main_set = set(w) # Unique vocabulary of data



In [60]:
# print(main_set) # Checking

In [28]:
# Calculating the frequency of words in the entire data.txt file
def counting_words(words):
	word_count = {}
	for word in words:
		if word in word_count:
			word_count[word] += 1
		else:
			word_count[word] = 1
	return word_count


In [29]:
# Calculating the probability of each word
def prob_cal(word_count_dict):
	probs = {}
	m = sum(word_count_dict.values())
	for key in word_count_dict.keys():
		probs[key] = word_count_dict[key] / m
	return probs


Now to create all types of different words possible, Following methods has to be used:

1.   Lemmatization
2.   Deletion of letter
3.   Switching Letter
4.   Replace Letter
5.   Insert new Letter



In [59]:
pip install pattern # To do Lemmatization

In [32]:
# LemmWord: extracting and adding
# root word i.e.Lemma using pattern module
import pattern
from pattern.en import lemma, lexeme
from nltk.stem import WordNetLemmatizer


def LemmWord(word): #This function splits each word from the given input string
	return list(lexeme(wd) for wd in word.split())[0] #lexeme function gives different forms of the word and return first element in the list


In [42]:
# Deleting letters from the words
def DeleteLetter(word):
	delete_list = [] # This list has stored the entire document in form of lists, removing the ith character from the original data.txt
	split_list = [] # list of tuple


	for i in range(len(word)): #For each index i, a tuple is created containing two parts:
	                        	#the substring from the start of the word to the i-th character (word[0:i]) and
	                         	#the substring from the i-th character to the end (word[i:]).
		split_list.append((word[0:i], word[i:]))


	for a, b in split_list: #loop iterates over each tuple (a, b) in split_list.
		delete_list.append(a + b[1:]) #For each tuple, it concatenates the first part a and the second part b
		                              #excluding its first character (b[1:]),
	#effectively removing the i-th character from the original word.
	return delete_list


In [41]:
print(DeleteLetter('sky')) # Checking

Splited list: [('', 'sky'), ('s', 'ky'), ('sk', 'y')]
Delete operation :
['ky', 'sy', 'sk']


In [35]:
# Switching two letters in a word
def Switch_(word):
	split_list = []
	switch_l = []

	#creating pair of the words(and breaking them)
	for i in range(len(word)):
		split_list.append((word[0:i], word[i:]))

	#Printint the first word (i.e. a)
	#then replacing the first and second character of b
	switch_l = [a + b[1] + b[0] + b[2:] for a, b in split_list if len(b) >= 2] #it switches the first and 2nd letter of word in b while keeping rest letters in b as it is
	return switch_l


In [36]:
def Replace_(word):
	split_l = []
	replace_list = []

	# Replacing the letter one-by-one from the list of alphs
	for i in range(len(word)):
		split_l.append((word[0:i], word[i:]))
	alphs = 'abcdefghijklmnopqrstuvwxyz'
	#For each tuple and each letter l in alphs, it concatenates the first  a,
	# the letter l, and the remaining part of b excluding its first letter
	replace_list = [a + l + (b[1:] if len(b) > 1 else '')
					for a, b in split_l if b for l in alphs] #The condition if b ensures that b is not empty.
	return replace_list


In [37]:
def insert_(word):
	split_l = []
	insert_list = []

	# Making pairs of the split words
	for i in range(len(word) + 1):
		split_l.append((word[0:i], word[i:]))

	#For each tuple and each letter l in alphs,
	#it concatenates the first part a, the letter l, and the second part b.
	alphs = 'abcdefghijklmnopqrstuvwxyz'
	insert_list = [a + l + b for a, b in split_l for l in alphs]
	return insert_list


In [38]:
# Collecting all the words
# in a set(so that no word will repeat)
def colab_1(word, allow_switches=True):
	colab_1 = set()
 #Applying all edit options
	colab_1.update(DeleteLetter(word))
	if allow_switches:
		colab_1.update(Switch_(word))
	colab_1.update(Replace_(word))
	colab_1.update(insert_(word))
	return colab_1

# collecting words using by allowing switches
def colab_2(word, allow_switches=True):
	colab_2 = set()
	#edit_one is set to the result of colab_1, containing variations of the original word after one level of edits.
	edit_one = colab_1(word, allow_switches=allow_switches)
 #For each word w in edit_one, the function generates further variations by applying colab_1 to w.
 #This effectively performs two levels of edits on the original word.
	for w in edit_one:
		if w:
			edit_two = colab_1(w, allow_switches=allow_switches)
			colab_2.update(edit_two)
	return colab_2


In [39]:
# Only storing those values which are in the vocab
def get_corrections(word, probs, vocab, n=2):
	suggested_word = []
	best_suggestion = []
	# Now suggested_word list contains all the possible variation of the word.
	# if a word is not in vocab then it creates level 1 variation with the intersection with
	# vocab and check and if it still does not create or find any valid word then it creates another level 2 variation of a word
	# with intersection with vocab and check if it get any valid word.
	suggested_word = list(
		(word in vocab and word) or colab_1(word).intersection(vocab)
		or colab_2(word).intersection(
			vocab))

	# finding out the words with high frequencies
	best_suggestion = [[s, probs[s]] for s in list(reversed(suggested_word))]
	return best_suggestion


In [59]:
# Input
my_word = input("Input the word : ")

print("The suggestions are: ")
print(end='\n')

# Counting word function
word_count = counting_words(main_set)

# Calculating probability
probs = prob_cal(word_count)

# only storing correct words
tmp_corrections = get_corrections(my_word, probs, main_set, 2)
for i, word_prob in enumerate(tmp_corrections):
	if(i < 3):
		print(word_prob[0])
	else:
		break


Input the word : masimub
The suggestions are: 

maximum
maximus
